In [ ]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats

import plotly
import plotly.graph_objs as go
import plotly.offline as offline
from plotly.colors import n_colors
import plotly.io as pio

pio.templates.default = "plotly_white"

import peptide_forest
pd.set_option("max_columns", 1000)
pd.set_option("max_rows", 50)

In [ ]:
tmts = ['126', '127L', '127H', '128L', '128H', '129L', '129H', '130L', '130H', '131L']

expected_values = {
    "126": (0, 1),
    "127L": (1, 1),
    "127H": (1, 0),
    "128L": (0.5, 1),
    "128H": (0.5, 0),
    "129L": (0.2, 1),
    "129H": (0.2, 0),
    "130L": (0.1, 1),
    "130H": (0.1, 0),
    "131L": (1, 1),
    #label: (EColi, Human)
}

# expected_values = {
#     "126": (0, 1),
#     "127L": (0.97, 1),
#     "127H": (1, 0),
#     "128L": (0.49, 1),
#     "128H": (0.4, 0),
#     "129L": (0.15, 1),
#     "129H": (0.15, 0),
#     "130L": (0.06, 1),
#     "130H": (0.07, 0),
#     "131L": (0.85, 1),
#     #label: (EColi, Human)
# }


# quant_df2 = pd.read_csv("../data/E13/_quant_new/04854_F1_R8_P0109699E13_TMT10_quant_pots.csv", index_col=0)
quant_df = pd.read_csv("../data/E32/04854_F1_R8_P0109699E32_TMT10_quant_pots.csv")
output = '../01Apr_E13.csv'
output = '../01Apr_E32.csv'
final_df = pd.read_csv(output)

In [ ]:
all_eng = [
    c.split("Score_processed_")[1] for c in final_df.columns if "Score_processed" in c
]

In [ ]:
quant_df.rename(columns={'spectrum_id': 'Spectrum ID'}, inplace=True)

In [ ]:
quant_df.head()

In [ ]:
quant_df.rename(
    columns={
        'spectrum_id': 'Spectrum ID',
    }, 
    inplace=True
)

In [ ]:
qdf = quant_df.pivot(index='Spectrum ID', columns='label', values='quant_value')

In [ ]:
for l1, l2 in itertools.combinations(qdf.columns, 2):
    qdf[f'log2({l1}/{l2})'] = np.log2(qdf[l1]/qdf[l2])

In [ ]:
qdf.head()

In [ ]:
qdf.loc[13211, :]

In [ ]:
qdf.describe()

In [ ]:
# Add species column
def determine_species(row):
    code = 0
    if "HUMAN" in row["Protein ID"]:
        code += 1  # 2^^0
    if "ECOLI" in row["Protein ID"]:
        code += 2  # 2^^1
    if "cont" in row["Protein ID"]:
        code += 4  # 2^^2
    return code

final_df["species code"] = final_df.apply(determine_species, axis=1)


In [ ]:
final_df.head()

In [ ]:
m = pd.merge(final_df, qdf, on="Spectrum ID", right_index=False).reset_index(drop=True)

In [ ]:
m.head()

In [ ]:
q_val_cuts = np.logspace(-4, -1, num=4)
sorted(q_val_cuts, reverse=True)

In [ ]:
for cut in sorted(q_val_cuts, reverse=True):
    for eng in all_eng:
        idx_below_cut_off_for_e = m[m[f'q-value_{eng}'] <= cut].index
        eng_per_q_col = f"top_target_{eng}_at_{cut}"
        m[eng_per_q_col] = False
        m.loc[idx_below_cut_off_for_e, eng_per_q_col] = True
            

In [ ]:
m[m[eng_per_q_col]].head()

# Version 1

## To visualize the data for on searchengine at one q-level

In [ ]:
mask = m['top_target_mascot_1_0_0_at_0.01'] & (m['species code'] == 2) & (~m['Is decoy'])
# mask = m['top_target_msfragger_20190222'] & (m['species code'] == 2)

In [ ]:
l_1 = '127L' # 1
l_2 = '130L' # 0.1
expected = np.log2(1/0.1)

In [ ]:
ion_sum = np.log10(1e7 / (m[mask][l_1] + m[mask][l_2]))
log_min = np.floor(ion_sum.min())
log_max = np.ceil(ion_sum.max())
labels = np.logspace(log_min, log_max, num=10)
ddf = pd.DataFrame(
    {
        'ion_sum' : ion_sum, 
        'cat' : pd.cut(ion_sum, bins=10, labels=labels),
        f'log2({l_1}/{l_2})' : np.log2(m[mask][l_1] / m[mask][l_2])
    }
)

In [ ]:
labels

In [ ]:
np.log10(labels)

In [ ]:
colors = n_colors('rgb(5, 200, 200)', 'rgb(200, 10, 10)', 10, colortype='rgb')

fig = go.Figure()
for color, (name, grp) in zip(colors, ddf.groupby('cat')):
# for data_line, color in zip(data, colors):
    fig.add_trace(
        go.Violin(
            x=grp[f'log2({l_1}/{l_2})'], 
            line_color=color,
            name="Bin {0:.3f}".format(np.log10(name))
        )
    )
    print(f'Group bin @ {np.log10(name)} has {grp.shape[0]} elements')
fig.add_shape(
    dict(
        type="line",
        x0=expected,
        y0=log_min,
        x1=expected,
        y1=log_max,
        line=dict(
            color="Black",
            width=1
        )
))
fig.update_traces(orientation='h', side='positive', width=3, points=False)
fig.update_layout(
    xaxis_showgrid=False, 
    xaxis_zeroline=False,
    xaxis_title=f'log2({l_1}/{l_2})', 
    yaxis_title=f'log10(1e7/({l_1} + {l_2}))')
fig.show()



# Version 2

## delta expected - observed as heatmaps

In [ ]:
q_val_cuts = np.logspace(-4, -1, num=4)
for l_1, l_2 in itertools.combinations(expected_values.keys(), 2):
    if 0 in expected_values[l_1] + expected_values[l_2]:
        continue

    m['ion_sum'] = np.log10(1e7 / (m[l_1] + m[l_2]))
    log_min = np.floor(m['ion_sum'].min())
    log_max = np.ceil(m['ion_sum'].max())
    labels = np.logspace(log_min, log_max, num=10)
    m['cat'] = pd.cut(m['ion_sum'], bins=10, labels=labels)

    expected = np.log2(expected_values[l_1][0] / expected_values[l_2][0])
    print(l_1, l_2, 'expected:',expected, expected_values[l_1], expected_values[l_2])
    
    for cut in sorted(q_val_cuts, reverse=True):
        z = []
        y = []
        hovertext = []

        for eng in all_eng:
            eng_per_q_col = f"top_target_{eng}_at_{cut}"
            mask = m[eng_per_q_col] & (m['species code'] == 2) & (~m['Is decoy'])
            values = []
            hovertext.append([])
            for name, grp in m[mask].groupby('cat'):
                values.append(expected - grp[f'log2({l_1}/{l_2})'].mean())
                hovertext[-1].append('n:{0}<br />delta mean:{1}<br />mean:{2}<br />'.format(
                    grp.shape[0],
                    expected - grp[f'log2({l_1}/{l_2})'].mean(),
                    grp[f'log2({l_1}/{l_2})'].mean()
                ))
            z.append(values)
            y.append(eng)

        fig = go.Figure(
            data=go.Heatmap(
                z=z,
                x=np.log10(labels),
                y=y,
                colorscale='rdylgn_r',
                text=hovertext,
                hoverinfo = "text"
            )
        )

        fig.update_layout(
            title=f'Difference to expected ratio of {expected:.3f} at q-values <= {cut}',
            xaxis_title=f'log10(1e7/({l_1}+{l_2}))', 
#             yaxis_title=f'log10(1e7/({l_1} + {l_2}))')
        )

        fig.show()


        break
    break

# Version 3

## Statistical evaluation

In [ ]:
# import scipy.stats

# q_val_cuts = np.logspace(-4, -2, num=3)
# print("Kruskal-Wallis H-test")
# for l_1, l_2 in itertools.combinations(expected_values.keys(), 2):
#     if 0 in expected_values[l_1] + expected_values[l_2]:
#         continue

#     m['ion_sum'] = np.log10(1e7 / (m[l_1] + m[l_2]))
#     log_min = np.floor(m['ion_sum'].min())
#     log_max = np.ceil(m['ion_sum'].max())
#     labels = np.logspace(log_min, log_max, num=10)
#     m['cat'] = pd.cut(m['ion_sum'], bins=10, labels=labels)
#     m.sort_values(['cat'], inplace=True)
    
#     expected = np.log2(expected_values[l_1][0] / expected_values[l_2][0])
#     print(l_1, l_2, 'expected:',expected, expected_values[l_1], expected_values[l_2])
    
#     for cut in sorted(q_val_cuts, reverse=False):
#         print(f"Q-value cut {cut}")
#         for category in sorted(m['cat'].unique()):
#             all_distributions = []
#             worthy = True
#             for eng in all_eng:
#                 eng_per_q_col = f"top_target_{eng}_at_{cut}"
#                 mask = m[eng_per_q_col] & (m['species code'] == 2) & (~m['Is decoy']) & (m['cat'] == category)
#                 if m[mask][f'log2({l_1}/{l_2})'].empty:
#                     continue
#                 if m[mask][f'log2({l_1}/{l_2})'].values.size < 5:
#                     continue
#                 all_distributions.append(
#                     np.copy(m[mask][f'log2({l_1}/{l_2})'].values)
#                 )
# #                 if all_distributions[-1].size < 5:
# #                     worthy = False
#             if len(all_distributions) < 3:
#                 worthy = False
#             if not worthy:
#                 print(f"Not enough samples at q-value {cut:.6f}, intensity bin {np.log10(category):.3f}")
#             else:
#                 statistic, pvalue = scipy.stats.kruskal(
#                     *all_distributions
#                 )
#                 print(f"At q-value {cut:.6f}, intensity bin {np.log10(category):.3f} gives p-value {pvalue:.3f}")
# #             if pvalue <= 0.01:
# #                 print("<")
# #                 for eng1, eng2 in itertools.combinations(all_eng, 2):
# #                     eng1_per_q_col = f"top_target_{eng1}_at_{cut}"
# #                     mask1 = m[eng1_per_q_col] & (m['species code'] == 2) & (~m['Is decoy']) & (m['cat'] == category)
# #                     eng2_per_q_col = f"top_target_{eng2}_at_{cut}"
# #                     mask2 = m[eng2_per_q_col] & (m['species code'] == 2) & (~m['Is decoy']) & (m['cat'] == category)

# #                     if m[mask1][f'log2({l_1}/{l_2})'].values.size < 20 or m[mask2][f'log2({l_1}/{l_2})'].values.size < 20 :
# #                         print(f"not enough samples to compare distributions from {eng1} & {eng2}")
# #                         continue
# #                     else:
# #                         statistic, pvalue = scipy.stats.mannwhitneyu(
# #                             m[mask1][f'log2({l_1}/{l_2})'].values,
# #                             m[mask2][f'log2({l_1}/{l_2})'].values
# #                         )
# #                         print(f">> Mann-Whitney-U p-value {eng1:>30} vs {eng2:30}: {pvalue:.4f}")

#                 break
#             break
#         break
#     break

In [ ]:
cut = 0.01
eng1 = "mascot_1_0_0"
eng2 = "RF-reg"
eng1_per_q_col = f"top_target_{eng1}_at_{cut}"
for category in sorted(m['cat'].unique()):
    mask1 = m[eng1_per_q_col] & (m['species code'] == 2) & (~m['Is decoy']) & (m['cat'] == category)
    eng2_per_q_col = f"top_target_{eng2}_at_{cut}"
    mask2 = m[eng2_per_q_col] & (m['species code'] == 2) & (~m['Is decoy']) & (m['cat'] == category)

    if m[mask1][f'log2({l_1}/{l_2})'].values.size < 5 or m[mask2][f'log2({l_1}/{l_2})'].values.size < 5 :
    #                     print(f"not enough samples to compare distributions from {eng1} & {eng2}")
    #                     continue
        pvalue = np.nan
    else:
    #                     funct = scipy.stats.mannwhitneyu
        funct = scipy.stats.ks_2samp
        statistic, pvalue = funct(
            m[mask1][f'log2({l_1}/{l_2})'].values,
            m[mask2][f'log2({l_1}/{l_2})'].values
        )
        print(f">> Kolmogorov-Smirnov {eng1:>9} vs {eng2:9} -log(p-value) : {-1*np.log(pvalue):.4f}")


In [ ]:
dfdf = []

q_val_cuts = np.logspace(-4, -2, num=3)
for l_1, l_2 in itertools.combinations(expected_values.keys(), 2):
    if 0 in expected_values[l_1] + expected_values[l_2]:
        continue

    m['ion_sum'] = np.log10(1e7 / (m[l_1] + m[l_2]))
    log_min = np.floor(m['ion_sum'].min())
    log_max = np.ceil(m['ion_sum'].max())
    labels = np.logspace(log_min, log_max, num=10)
    m['cat'] = pd.cut(m['ion_sum'], bins=10, labels=labels)
    m.sort_values(['cat'], inplace=True)
    
    expected = np.log2(expected_values[l_1][0] / expected_values[l_2][0])
    print(l_1, l_2, 'expected:',expected, expected_values[l_1], expected_values[l_2])
    
    for cut in sorted(q_val_cuts, reverse=False):
        
        for category in sorted(m['cat'].unique()):
#             print(f"Q-value cut {cut} and cat", np.log10(category))
            all_distributions = []
            worthy = True
#             print("Category:", np.log10(category))
            for eng1, eng2 in itertools.combinations(all_eng, 2):
                eng1_per_q_col = f"top_target_{eng1}_at_{cut}"
                mask1 = m[eng1_per_q_col] & (m['species code'] == 2) & (~m['Is decoy']) & (m['cat'] == category)
                eng2_per_q_col = f"top_target_{eng2}_at_{cut}"
                mask2 = m[eng2_per_q_col] & (m['species code'] == 2) & (~m['Is decoy']) & (m['cat'] == category)

                if m[mask1][f'log2({l_1}/{l_2})'].values.size < 5 or m[mask2][f'log2({l_1}/{l_2})'].values.size < 5 :
#                     print(f"not enough samples to compare distributions from {eng1} & {eng2}")
#                     continue
                    pvalue = np.nan
                else:
#                     funct = scipy.stats.mannwhitneyu
                    funct = scipy.stats.ks_2samp
                    statistic, pvalue = funct(
                        m[mask1][f'log2({l_1}/{l_2})'].values,
                        m[mask2][f'log2({l_1}/{l_2})'].values
                    )
                
#                     print(f">> Kolmogorov-Smirnov {eng1:>30} vs {eng2:30} p-value : {pvalue:.4f}")
                    
                dfdf.append(
                    {
                        "comparison" : f"{eng1}_vs_{eng2}",
                        "pvalue": pvalue,
                        "qvalue_cut": cut,
                        "cat": np.log10(category),
                        "ratio": f'log2({l_1}/{l_2})'
                    }
                )
#         break
#     break

In [ ]:
dfdf = pd.DataFrame(dfdf)

In [ ]:
dfdf.head()

In [ ]:
dfdf['-log(p-value)'] = -1 * np.log10(dfdf['pvalue'])
dfdf['comparison_at_qvalue'] = dfdf['qvalue_cut'].astype(str) + "_@_" + dfdf['comparison'] + "//" + dfdf['ratio']

In [ ]:
dfdf.sort_values(["comparison_at_qvalue"], inplace=True, ascending=False)

In [ ]:
for name, grp in dfdf.groupby(["qvalue_cut", "ratio"]):
    fig = go.Figure(
        data=go.Heatmap(
            z=grp['-log(p-value)'],
#             z=grp['pvalue'],
            x=grp['cat'],
            y=grp['comparison'],
            colorscale=[[0, "rgb(166,206,227)"],
                [0.25, "rgb(31,120,180)"],
                [0.45, "rgb(178,223,138)"],
                [0.65, "rgb(51,160,44)"],
                [0.85, "rgb(251,154,153)"],
                [1, "rgb(227,26,28)"]],
            colorbar=dict(
                title="-log(p-value)",
                titleside="top",
                tickmode="array",
                tickvals=[-1*np.log10(0.05), -1*np.log10(0.01), -1*np.log10(0.001)],
                ticktext=["*: -log(0.05)", "**: -log(0.01)", "***: -log(0.001)"],
                ticks="outside"
            )
        )
    )
    fig.update_layout(
        title=f"@{name}"
    )
    fig.show()

In [ ]:
-1 * np.log10(0.01)

In [ ]:
dfdf.sort_values(['-log(p-value)'], ascending=False).head()

In [ ]:
dfdf.head()

## Violin plots for all engines

In [ ]:
q_val_cuts = np.logspace(-4, -2, num=3)
for l_1, l_2 in itertools.combinations(expected_values.keys(), 2):
    if 0 in expected_values[l_1] + expected_values[l_2]:
        continue

    m['ion_sum'] = np.log10(1e7 / (m[l_1] + m[l_2]))
    log_min = np.floor(m['ion_sum'].min())
    log_max = np.ceil(m['ion_sum'].max())
    labels = np.logspace(log_min, log_max, num=10)
    m['cat'] = pd.cut(m['ion_sum'], bins=10, labels=labels)
    m.sort_values(['cat'], inplace=True)
    
    expected = np.log2(expected_values[l_1][0] / expected_values[l_2][0])
    print(l_1, l_2, 'expected:',expected, expected_values[l_1], expected_values[l_2])
    
    for cut in sorted(q_val_cuts, reverse=True):
        z = []
        y = []
        fig = go.Figure()
        fig.add_shape(
            dict(
                type="line",
                x0=log_min,
                y0=expected,
                x1=log_max,
                y1=expected,
                line=dict(
                    color="grey",
                    width=1
                )
        ))
        for eng in all_eng:
            eng_per_q_col = f"top_target_{eng}_at_{cut}"
            mask = m[eng_per_q_col] & (m['species code'] == 2) & (~m['Is decoy'])
            
            fig.add_trace(
                go.Violin(
                    x = np.log10(m[mask]['cat'].astype(float)), # intensity log sum bins
                    y = m[mask][f'log2({l_1}/{l_2})'], # log2 ratios
                    legendgroup=eng, 
                    scalegroup=eng, 
                    name='{0} n:{1}'.format(eng, len(m[mask])),

                )
            )
        
        fig.update_layout(
            title=f'Expected ratio of {expected:.3f} at q-values <= {cut}',
            xaxis_title=f'log10(1e7/({l_1}+{l_2}))', 
            yaxis_title=f'log2({l_1}/{l_2})'
        )
        fig.update_traces(
            box_visible=True, 
            meanline_visible=True,
            points='all', # show all points
            jitter=0.25,  # add some jitter on points for better visibility
            scalemode='count'
        )
        fig.update_layout(violinmode='group')
        fig.show()


        break
#     break

# playground below

In [ ]:
_d = m[mask]

data = go.Scattergl(
    x=np.log10(1e7 / (_d[l_1] + _d[l_2])), 
    y=_d[f'log2({l_1}/{l_2})'],
    mode='markers',
#     marker=dict(
#         size=3,
#         color=df.loc[subset.index, c],
#         colorscale='rdylgn',
#         showscale=True
#     ),
#     hovertext = "Q-values<br />" + \
#     "Mascot:    " + df.loc[subset.index, "q-value_mascot_1_0_0"].astype(str) + "<br />" + \
#     "OMSSA:     " + df.loc[subset.index, "q-value_omssa_2_1_9"].astype(str) + "<br />" + \
#     "MSGF+:     " + df.loc[subset.index, "q-value_msgfplus_v2018_06_28"].astype(str) + "<br />" + \
#     "X!Tandem:  " + df.loc[subset.index, "q-value_xtandem_vengeance"].astype(str) + "<br />" + \
#     "MSFragger: " + df.loc[subset.index, "q-value_msfragger_20190222"].astype(str) + "<br />" + \
#     "RF:        " + df.loc[subset.index, "q-value_RF-reg"].astype(str) + "<br />" + \
#     "Spectrum ID: " + df.loc[subset.index, 'Spectrum ID'].astype(str)
#     ,
#     hoverinfo = "text"
)
# data.on_click(copy_to_clipboard)
fig = go.Figure(
    data=data
)

fig.update_layout(template="plotly_white", title=">>")
fig.show()

# Seaborn below

In [ ]:
#quant_df["MSMS_ID"] = quant_df["MSMS_ID"].str.lstrip("F0")
quant_df["MSMS_ID"] = quant_df["spectrum_id"]
unique_spec_ids = final_df["Spectrum ID"].drop_duplicates()
ma_df = pd.DataFrame(index=unique_spec_ids)

q_val_cuts = sorted([float(f"1e-{j}") for j in np.arange(4, 1, -1)]) + [1e-1]

# final_df = peptide_forest.results.calc_all_final_q_vals(
#     df=final_df, frac_tp=0.9, top_psm_only=True, initial_engine=None
# )

all_eng = [
    c.split("Score_processed_")[1] for c in final_df.columns if "Score_processed" in c
]
for cut in q_val_cuts:
    for eng in all_eng:
        eng_per_q_col = f"top_target_{eng}_at_{cut}"
        target_col = f"top_target_{eng}"
        marked_targets = peptide_forest.results.mark_top_targets(
            df=final_df, q_cut=cut
        )[[target_col, "Spectrum ID"]]
        marked_targets = marked_targets[marked_targets[target_col]]["Spectrum ID"]
        ma_df[eng_per_q_col] = False
        ma_df.loc[marked_targets, eng_per_q_col] = True


In [ ]:
# Drop rows that never appear as top target
top_target_cols = [c for c in ma_df.columns if "top_target" in c]
ma_df[top_target_cols] = ma_df[top_target_cols].astype(bool)
ma_df = ma_df[ma_df[top_target_cols].any(axis=1)]

In [ ]:
# Generate all and any engines
for cut in q_val_cuts:
    eng_cols_per_cut = [f"top_target_{engine}_at_{cut}" for engine in all_eng]

    ma_df[f"top_target_any_engine_at_{cut}"] = False
    ma_df.loc[
        ma_df[eng_cols_per_cut].any(axis=1), f"top_target_any_engine_at_{cut}"
    ] = True

    ma_df[f"top_target_all_engines_at_{cut}"] = False
    ma_df.loc[
        ma_df[eng_cols_per_cut].all(axis=1), f"top_target_all_engines_at_{cut}"
    ] = True

In [ ]:
all_eng = all_eng + ["all_engines", "any_engine"]

for t in tmts:
    values = quant_df[quant_df["label"] == t][["MSMS_ID", "quant_value"]].astype({"MSMS_ID": "int64", "quant_value": "float64"})
    # Remove missing spectra
    values = values[values["MSMS_ID"].isin(ma_df.index)]
    ma_df.loc[values["MSMS_ID"], t] = values["quant_value"].to_list()


In [ ]:
# Remove all SpecIDs where quant value is 0 in a mixed column
mixed_cols = ["127L", "128L", "129L", "130L", "131L"]
ma_df = ma_df[~ma_df[mixed_cols].any(axis=1) == 0]

# Remove all nan rows
ma_df = ma_df[~ma_df[tmts].isna().all(axis=1)]

In [ ]:
quotients = list(itertools.combinations(tmts, 2))

# [TRISTAN] temp
all_eng = ["all_engines", "any_engine", "RF-reg", "omssa_2_1_9"]

In [ ]:
for ratio in quotients:
    for species in [2, 4]:
        if (
            species == "E_coli"
            and expected_values[ratio[1]][0] != 0
            and expected_values[ratio[0]][0] != 0
        ):
            exp_y = np.log2(expected_values[ratio[0]][0] / expected_values[ratio[1]][0])
        elif (
            species == "H_sapiens"
            and expected_values[ratio[1]][1] != 0
            and expected_values[ratio[0]][1] != 0
        ):
            exp_y = np.log2(expected_values[ratio[0]][1] / expected_values[ratio[1]][1])
        else:
            continue
        species_df = ma_df[ma_df["species"] == species]
        for cut in q_val_cuts:
            plt.figure()
            cols_oi = [f"top_target_{eng}_at_{cut}" for eng in all_eng]
            df_plot = pd.DataFrame()

            for c in cols_oi:
                if species_df[c].sum() <= 1:
                    continue

                sub_df = species_df[species_df[c]]
                # Get around RuntimeWarning for log2
                sub_df = sub_df.replace(to_replace=0, value=1e-308)
                df_eng = pd.DataFrame()
                df_eng["x_axis"] = 1e7 / (sub_df[ratio[0]] + sub_df[ratio[1]])
                df_eng["y_axis"] = np.log2(sub_df[ratio[0]] / sub_df[ratio[1]])
                df_eng["engine"] = c.split("top_target_")[1].split("_at")[0]
                df_eng = df_eng[df_eng["x_axis"] < 2]

                if len(df_plot) == 0:
                    df_plot = df_eng
                else:
                    df_plot = pd.concat([df_eng, df_plot])

            plot = sns.lmplot(
                x="x_axis",
                y="y_axis",
                hue="engine",
                data=df_plot,
                x_bins=np.logspace(-2, 1, 20),
                scatter_kws={"s": 20, "alpha": 0.5},
                line_kws={"lw": 2},
                x_ci="sd",
                markers="x",
                legend=False,
            )
            err_lines = [
                l
                for l in plot.ax.lines
                if len(l.get_xdata()) == 2 and l.get_xdata()[0] == l.get_xdata()[1]
            ]
            for el in err_lines:
                el.set_linewidth(1)
                x_caps = el.get_xdata()[0]
                y_caps = el.get_ydata()
                position = {
                    "ha": "center",
                    "va": "center",
                    "size": 20,
                    "color": el.get_color(),
                }
                cap_lower = plot.ax.annotate("-", xy=(x_caps, y_caps[0]), **position)
                cap_upper = plot.ax.annotate("-", xy=(x_caps, y_caps[1]), **position)

            legend = plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.2), ncol=3)
            plot.ax.axhline(
                exp_y, color="black", linestyle="--", linewidth=2, alpha=0.5
            )
            plot.ax.set_xlim(0.01, 2)
            plot.set_axis_labels(
                f"1e7 / sum {ratio[0]}+{ratio[1]}", f"log2 {ratio[0]}x{ratio[1]}"
            )
            plot.set(xscale="log")
            plot.fig.suptitle(f"{ratio[0]}_{ratio[1]}_{species}_at_{cut}")
            plt.savefig(
                f"../plots/ma/{ratio}_{species}_at_{cut}.png",
                bbox_extra_artists=[legend],
                bbox_inches="tight",
                dpi=600,
            )
            plt.show()